# Análise exploratória e teste amostral - linha LECD50

**TODO:**
- (1) Tabela pré-reprocessamento
- (2) Gráficos da análise exploratória
- (3) Tabela pós-reprocessamento

Preparar o ambiente:

In [60]:

#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent
parent_path = current_path.parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = parent_path / 'scripts' 

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
 
from graphs import *
from categorize_trips import *
from circular_trips import *
from gps_data import *
from treat_data import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


# (1) Tabela pré-reprocessamento

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

#### 1 - Importar e tratar os dados da amostra

In [61]:
amostra = pd.read_excel('../data/raw/Rio Onibus - viagens LECD50 e 605 com Novo Shape.xlsx')

# converter para datatime
meses = {
    'jan.': 'January',
    'fev.': 'February',
    'mar.': 'March',
    'abr.': 'April',
    'mai.': 'May',
    'jun.': 'June',
    'jul.': 'July',
    'ago.': 'August',
    'set.': 'September',
    'out.': 'October',
    'nov.': 'November',
    'dez.': 'December'
}

# Função para converter as datas/horários no formato especificado
def converter_datahora(data_texto):
    for mes_abreviado, mes_completo in meses.items():
        data_texto = data_texto.replace(mes_abreviado, mes_completo)
    return pd.to_datetime(data_texto, format="%d de %B de %Y, %H:%M:%S")

# Aplicar a função de conversão às colunas 'datetime_partida' e 'datetime_chegada'
amostra['datetime_partida'] = amostra['datetime_partida'].apply(converter_datahora)
amostra['datetime_chegada'] = amostra['datetime_chegada'].apply(converter_datahora)
amostra['data'] = amostra['datetime_partida'].dt.date

# setar tipos de dados 
amostra['servico'] = amostra['servico'].astype(str)
# amostra['datetime_partida'] = amostra['datetime_partida'].astype(str)
# amostra['datetime_chegada'] = amostra['datetime_chegada'].astype(str)
amostra['data'] = amostra['data'].astype(str)
amostra['id_veiculo'] = amostra['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
amostra['status'] = np.nan
amostra_tratada = amostra.copy()
amostra_tratada

,servico,id_veiculo,sentido,datetime_partida,datetime_chegada,data,status
0,LECD50,27565,C,2023-02-01 06:25:51,2023-02-01 07:04:21,2023-02-01,NaN
1,LECD50,27565,C,2023-02-01 07:26:51,2023-02-01 08:12:21,2023-02-01,NaN
2,LECD50,27565,C,2023-02-01 08:29:51,2023-02-01 09:16:51,2023-02-01,NaN
3,LECD50,27565,C,2023-02-01 09:40:21,2023-02-01 10:24:21,2023-02-01,NaN
4,LECD50,27565,C,2023-02-01 10:43:21,2023-02-01 11:26:51,2023-02-01,NaN
...,...,...,...,...,...,...,...
976,605,27632,C,2023-09-28 07:07:43,2023-09-28 07:53:43,2023-09-28,NaN
977,605,27632,C,2023-09-28 08:11:13,2023-09-28 08:46:43,2023-09-28,NaN
978,605,27632,C,2023-09-28 09:00:43,2023-09-28 09:49:13,2023-09-28,NaN
979,605,27632,C,2023-09-28 10:00:13,2023-09-28 10:42:13,2023-09-28,NaN


In [62]:
servicos = query_values(amostra_tratada, 'servico')
datas = amostra_tratada['data'].unique()
veiculos = amostra_tratada['id_veiculo'].unique()
print("Serviços:", servicos)
print("A quantidade total de viagens na amostra é:", amostra_tratada.shape[0])
print("A amostra é referente aos dias:", datas)
print("A amostra tem dados dos seguintes veículos:", veiculos)

Serviços: 'LECD50','605'
A quantidade total de viagens na amostra é: 981
A amostra é referente aos dias: ['2023-02-01' '2023-02-02' '2023-02-03' '2023-02-06' '2023-02-07'
 '2023-02-08' '2023-02-09' '2023-02-10' '2023-02-13' '2023-02-14'
 '2023-02-15' '2023-02-16' '2023-02-23' '2023-02-24' '2023-02-27'
 '2023-02-28' '2023-03-01' '2023-03-02' '2023-03-03' '2023-03-06'
 '2023-03-07' '2023-03-08' '2023-03-09' '2023-03-10' '2023-03-13'
 '2023-03-14' '2023-03-15' '2023-03-16' '2023-03-17' '2023-03-20'
 '2023-03-21' '2023-03-22' '2023-03-23' '2023-03-24' '2023-03-27'
 '2023-04-03' '2023-04-04' '2023-04-05' '2023-04-10' '2023-04-11'
 '2023-04-12' '2023-04-13' '2023-04-14' '2023-04-25' '2023-04-26'
 '2023-04-27' '2023-04-28' '2023-05-03' '2023-05-04' '2023-05-05'
 '2023-05-08' '2023-05-09' '2023-05-10' '2023-05-11' '2023-05-12'
 '2023-05-15' '2023-05-16' '2023-05-17' '2023-05-18' '2023-05-19'
 '2023-05-22' '2023-05-23' '2023-05-24' '2023-05-25' '2023-05-26'
 '2023-05-29' '2023-05-30' '2023-05-3

#### 2 - Remover viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [63]:
amostra_deduplicada = remove_overlapping_trips(amostra_tratada)
amostra_deduplicada

,servico,id_veiculo,sentido,datetime_partida,datetime_chegada,data,status
0,LECD50,27565,C,2023-02-01 06:25:51,2023-02-01 07:04:21,2023-02-01,NaN
1,LECD50,27565,C,2023-02-01 07:26:51,2023-02-01 08:12:21,2023-02-01,NaN
2,LECD50,27565,C,2023-02-01 08:29:51,2023-02-01 09:16:51,2023-02-01,NaN
3,LECD50,27565,C,2023-02-01 09:40:21,2023-02-01 10:24:21,2023-02-01,NaN
4,LECD50,27565,C,2023-02-01 10:43:21,2023-02-01 11:26:51,2023-02-01,NaN
...,...,...,...,...,...,...,...
976,605,27632,C,2023-09-28 07:07:43,2023-09-28 07:53:43,2023-09-28,NaN
977,605,27632,C,2023-09-28 08:11:13,2023-09-28 08:46:43,2023-09-28,NaN
978,605,27632,C,2023-09-28 09:00:43,2023-09-28 09:49:13,2023-09-28,NaN
979,605,27632,C,2023-09-28 10:00:13,2023-09-28 10:42:13,2023-09-28,NaN


In [64]:
count_non_nan = amostra_deduplicada['status'].notna().sum()
count_non_nan

0

### 3 Comparar dados da amostra com os dados das viagens completas

In [65]:
datas = query_values(amostra_deduplicada, 'data') 
id_veiculo = query_values(amostra_deduplicada, 'id_veiculo') 

viagem_completa = query_viagem_completa(datas, id_veiculo)


    SELECT
      data,
      SUBSTRING(id_veiculo, 2) as id_veiculo,
      servico_informado,
      sentido,
      datetime_partida,
      datetime_chegada
    FROM
      `rj-smtr.projeto_subsidio_sppo.viagem_completa`
    WHERE 
      data IN ('2023-02-01','2023-02-02','2023-02-03','2023-02-06','2023-02-07','2023-02-08','2023-02-09','2023-02-10','2023-02-13','2023-02-14','2023-02-15','2023-02-16','2023-02-23','2023-02-24','2023-02-27','2023-02-28','2023-03-01','2023-03-02','2023-03-03','2023-03-06','2023-03-07','2023-03-08','2023-03-09','2023-03-10','2023-03-13','2023-03-14','2023-03-15','2023-03-16','2023-03-17','2023-03-20','2023-03-21','2023-03-22','2023-03-23','2023-03-24','2023-03-27','2023-04-03','2023-04-04','2023-04-05','2023-04-10','2023-04-11','2023-04-12','2023-04-13','2023-04-14','2023-04-25','2023-04-26','2023-04-27','2023-04-28','2023-05-03','2023-05-04','2023-05-05','2023-05-08','2023-05-09','2023-05-10','2023-05-11','2023-05-12','2023-05-15','2023-05-16','2023-05-17',

Downloading: 100%|██████████| 31426/31426 [00:04<00:00, 6603.50rows/s]


Execução de query de viagens completas: sucesso.


Esta etapa cruza dados da amostra com as viagens completas para o serviço LECD50 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no amostra.

In [66]:
viagem_completa = treat_trips(viagem_completa)
amostra_classificada_completa = check_trips(amostra_deduplicada, viagem_completa, "Viagem identificada e já paga")
print(amostra_classificada_completa.shape[0])
amostra_classificada_completa.to_excel('./../data/treated/viagem_completa.xlsx')

Não existem casos duplicados no cruzamento de dados.
981


### 4 Comparar dados da amostra com os dados das viagens conformidade

In [67]:
viagem_conformidade = query_viagem_conformidade(datas, id_veiculo)


  SELECT
    data,
    SUBSTRING(id_veiculo, 2) as id_veiculo,
    servico_informado,
    sentido,
    datetime_partida,
    datetime_chegada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
  WHERE 
    data IN ('2023-02-01','2023-02-02','2023-02-03','2023-02-06','2023-02-07','2023-02-08','2023-02-09','2023-02-10','2023-02-13','2023-02-14','2023-02-15','2023-02-16','2023-02-23','2023-02-24','2023-02-27','2023-02-28','2023-03-01','2023-03-02','2023-03-03','2023-03-06','2023-03-07','2023-03-08','2023-03-09','2023-03-10','2023-03-13','2023-03-14','2023-03-15','2023-03-16','2023-03-17','2023-03-20','2023-03-21','2023-03-22','2023-03-23','2023-03-24','2023-03-27','2023-04-03','2023-04-04','2023-04-05','2023-04-10','2023-04-11','2023-04-12','2023-04-13','2023-04-14','2023-04-25','2023-04-26','2023-04-27','2023-04-28','2023-05-03','2023-05-04','2023-05-05','2023-05-08','2023-05-09','2023-05-10','2023-05-11','2023-05-12','2023-05-15','2023-05-16','2023-05-17','2023-05-18','2023

Downloading: 100%|██████████| 30741/30741 [00:04<00:00, 7164.05rows/s]


Execução de query de viagens conformidade: sucesso.


In [68]:
viagem_conformidade = treat_trips(viagem_conformidade)
amostra_classificada_conformidade = check_trips(amostra_classificada_completa, viagem_conformidade, "Viagem não atendeu os critérios de conformidade")
print(amostra_classificada_conformidade.shape[0])

Não existem casos duplicados no cruzamento de dados.
983


Existe dois casos em que o mesmo veículo teve 2 viagens na tabela de conformidade no mesmo horário para serviços diferentes (LECD50 e 433) e que não atenderam os critérios. As linhas duplicadas serão removidas para manter a mesma quantidade de viagens da amostra.

In [69]:
duplicated_data = amostra_classificada_conformidade[amostra_classificada_conformidade.duplicated(subset=['id_veiculo_amostra', 'datetime_partida_amostra'], keep=False)]
duplicated_data

,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,data,status,data_apurado,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
519,LECD50,27565,C,2023-02-06 06:19:26,2023-02-06 06:57:26,2023-02-06,Viagem não atendeu os critérios de conformidad...,2023-02-06,27565,433,I,2023-02-06 06:18:26,2023-02-06 09:36:56
520,LECD50,27565,C,2023-02-06 06:19:26,2023-02-06 06:57:26,2023-02-06,Viagem não atendeu os critérios de conformidade,2023-02-06,27565,LECD50,C,2023-02-06 06:19:26,2023-02-06 06:57:26
905,LECD50,27684,C,2023-04-27 09:01:17,2023-04-27 09:48:17,2023-04-27,Viagem não atendeu os critérios de conformidad...,2023-04-27,27684,433,I,2023-04-27 08:55:17,2023-04-27 12:18:47
906,LECD50,27684,C,2023-04-27 09:01:17,2023-04-27 09:48:17,2023-04-27,Viagem não atendeu os critérios de conformidade,2023-04-27,27684,LECD50,C,2023-04-27 09:01:17,2023-04-27 09:48:17


In [70]:
amostra_classificada_conformidade = amostra_classificada_conformidade[amostra_classificada_conformidade['servico_apurado'] != '433']
amostra_classificada_conformidade.to_excel('./../data/treated/viagem_conformidade.xlsx')

### 5 Verificar a existência de viagens circulares 

In [71]:
viagens_circulares = circular_trips(amostra_classificada_conformidade, viagem_completa, viagem_conformidade)
viagens_circulares.to_excel('./../data/treated/viagem_circular.xlsx')

Verificando se existem linhas circulares.


Downloading: 100%|██████████| 300/300 [00:00<00:00, 771.23rows/s]


Execução de query de viagens planejada: sucesso.
    servico_amostra id_veiculo_amostra sentido_amostra  \
422          LECD50              27565               C   
423          LECD50              27565               C   
424          LECD50              27565               C   
425          LECD50              27565               C   
426          LECD50              27565               C   
..              ...                ...             ...   
479          LECD50              27556               C   
480          LECD50              27682               C   
481          LECD50              27682               C   
482          LECD50              27556               C   
483          LECD50              27556               C   

    datetime_partida_amostra datetime_chegada_amostra        data status  \
422      2023-02-09 07:13:30      2023-02-09 07:52:00  2023-02-09    NaN   
423      2023-02-09 08:11:00      2023-02-09 08:52:00  2023-02-09    NaN   
424      2023-02-09 09:11:

Foram identificadas 921 das 981 viagens válidas da amostra, considerando que a LECD50 não teve viagens planejadas ou sinal de GPS após 2023-05-31 e para as datas da amostra de 2023 foram usados os dados da linha 605. Nestes casos, o veículo que realizou a viagem é o mesmo veículo que consta no amostra. 

A linha não teve viagem planejada em junho de 2022. Elas começaram em 16-07-2022.

Em julho, as viagens completas foram identificadas apenas nos dias 25, 26, 27 e 28, mesmo assim o POD foi zero no mês (explicação para o gráfico do POD).

### 6 Verificar dados de GPS

In [72]:
viagens_gps_classificadas = gps_data(viagens_circulares)

Downloading: 100%|██████████| 16873/16873 [00:01<00:00, 11293.75rows/s]

Execução de query de posições de GPS: sucesso.
Acesso aos sinais de GPS concluído com sucesso.
Tratamento de dados de GPS concluído com sucesso.


In [73]:
viagens_gps_classificadas.to_excel('./../data/treated/viagens_gps_classificadas.xlsx')

Checar se passa perto do ponto inicial e final

In [74]:
viagens_gps_classificadas_start_end = check_start_end_gps(viagens_gps_classificadas)

Downloading: 100%|██████████| 16/16 [00:00<00:00, 36.61rows/s]


Execução de query de viagens planejada: sucesso.


Downloading: 100%|██████████| 16873/16873 [00:01<00:00, 10672.28rows/s]


Execução de query de posições de GPS: sucesso.
1655
     servico_amostra id_veiculo_amostra sentido_amostra  \
0             LECD50              27565               C   
1             LECD50              27565               C   
2             LECD50              27565               C   
3             LECD50              27565               C   
4             LECD50              27565               C   
...              ...                ...             ...   
1650          LECD50              27682               C   
1651          LECD50              27556               C   
1652          LECD50              27556               C   
1653          LECD50              27556               C   
1654          LECD50              27556               C   

     datetime_partida_amostra datetime_chegada_amostra        data  \
0         2023-02-03 07:12:55      2023-02-03 07:57:25  2023-02-03   
1         2023-02-06 17:19:26      2023-02-06 18:09:26  2023-02-06   
2         2023-02-08 07:12:42

In [75]:
viagens_gps_classificadas_start_end.to_excel('./../data/treated/viagens_gps_classificadas_start_end.xlsx')

In [76]:
contar_nan = viagens_gps_classificadas_start_end['status'].notna().sum()
print(f"Tamanho da amostra: {amostra_tratada.shape[0]}")

print(f"Viagens classificadas: {contar_nan}")
print(f"Viagens classificadas em %: {round(contar_nan / amostra.shape[0],2)*100} %")

print(viagens_gps_classificadas_start_end['status'].value_counts())

Tamanho da amostra: 981
Viagens classificadas: 981
Viagens classificadas em %: 100.0 %
Viagem não atendeu os critérios de conformidade                              495
Viagem identificada e já paga                                                422
O veículo não passou no raio de 500m do ponto de partida/final do trajeto     62
Serviço não planejado para o dia                                               2
Name: status, dtype: int64


In [77]:
viagens_gps_classificadas_start_end.to_excel('./../data/output/analise_amostra_pre_solucao.xlsx')

# Exemplos (mapas)

Viagem não identificada em 2022 - não passou no raio de 500m na volta.

Veículo: A27684 - LECD50

partida: 2022-09-13 06:05:00	
chegada: 2022-09-13 06:51:00	

In [78]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-13"
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.45rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [79]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2022-09-13"
  AND servico = 'LECD50'
  AND timestamp_gps BETWEEN '2022-09-13 06:05:00'
  AND '2022-09-13 06:51:00'
  AND id_veiculo = 'A27684'
"""
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 68/68 [00:00<00:00, 157.41rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           68 non-null     object        
 1   servico              68 non-null     object        
 2   timestamp_gps        68 non-null     datetime64[ns]
 3   posicao_veiculo_geo  68 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.2+ KB


In [80]:
map = create_trip_map(gps_identificado, shape_identificado)
map.save('./../data/figures/maps/A27684_LECD50_13_09_2022.html')
map

Viagem não identificada em 2023 - não atendeu a conformidade.

Veículo: A27515 - LECD50

partida: 2023-03-27 06:24:00	
chegada: 2023-03-27 06:57:00	

In [81]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-03-27"
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.18rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [82]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2023-03-27"
  AND servico = 'LECD50'
  AND timestamp_gps BETWEEN '2023-03-27 06:24:00'
  AND '2023-03-27 06:57:00'
  AND id_veiculo = 'A27515'
"""
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 67/67 [00:00<00:00, 179.62rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           67 non-null     object        
 1   servico              67 non-null     object        
 2   timestamp_gps        67 non-null     datetime64[ns]
 3   posicao_veiculo_geo  67 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.2+ KB


In [83]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/A27684_LECD50_13_09_2022.html')
map

## (2) Gráficos da análise exploratória / análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [84]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico IN ({servicos})
  AND DATA BETWEEN '2022-06-01' AND '2023-10-15'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 369/369 [00:00<00:00, 878.55rows/s]


In [85]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [86]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviços: {}".format(servicos), font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

In [87]:
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])

# Criar o gráfico de linha com Plotly Express
fig = px.line(pod_pre_reprocessamento, 
              x='data', 
              y='perc_km_planejada', 
              title='Percentual de KM Planejada ao Longo do Tempo', 
              labels={'data': 'Data', 'perc_km_planejada': 'Percentual de KM Planejada'}, 
              markers=True)

# Mostrar o gráfico
fig.show()

Em dezembro as viagens são identificadas corretamente. Vale ressaltar que neste mês houveram viagens apenas até o dia 14 de dezembro (confirmado via dados de GPS).

In [88]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-12-14" 
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.77rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [89]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-12-14"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2022-12-14T14:00:00"
  AND "2022-12-14T14:42:00"
  AND id_veiculo = "A27684"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 56/56 [00:00<00:00, 134.62rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           56 non-null     object        
 1   servico              56 non-null     object        
 2   timestamp_gps        56 non-null     datetime64[ns]
 3   posicao_veiculo_geo  56 non-null     object        
 4   status_viagem        56 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.3+ KB


In [90]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

Com a mudança do shape no dia 01/01/2023, a viagem passou a não ser mais identificada corretamente. O problema persiste até hoje. 

Março de 2023:

Viagem encontrada no período afetado:
Mesmo as viagens identificadas, foram identificadas de forma errada.

In [91]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-03-22" 
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.43rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [92]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-03-22"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2023-03-22T09:49:00"
  AND "2023-03-22T10:40:00"
  AND id_veiculo = "A27556"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 281/281 [00:00<00:00, 631.46rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           281 non-null    object        
 1   servico              281 non-null    object        
 2   timestamp_gps        281 non-null    datetime64[ns]
 3   posicao_veiculo_geo  281 non-null    object        
 4   status_viagem        281 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 11.1+ KB


In [93]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

## Análise das viagens da amostra x apuradas (pós-solução)

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pos_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao_solucao	datetime_chegada_apuracao_solucao	status


Tabela com as viagens planejadas com o shape correto: `rj-smtr-dev.SMTR202303002183_reprocessamento_planejado.viagem_planejada`

Dataset com dados reprocessados: `rj-smtr-dev.SMTR202303002183_reprocessamento`

In [94]:
viagem_completa_reprocessada = query_viagem_completa(data = datas, id_veiculo = id_veiculo, reprocessed=True,
                                                     reprocessed_table='rj-smtr-dev.SMTR202303002183_reprocessamento.viagem_completa')


    SELECT
      data,
      SUBSTRING(id_veiculo, 2) as id_veiculo,
      servico_informado,
      sentido,
      datetime_partida,
      datetime_chegada
    FROM
      rj-smtr-dev.SMTR202303002183_reprocessamento.viagem_completa
    WHERE 
      data IN ('2023-02-01','2023-02-02','2023-02-03','2023-02-06','2023-02-07','2023-02-08','2023-02-09','2023-02-10','2023-02-13','2023-02-14','2023-02-15','2023-02-16','2023-02-23','2023-02-24','2023-02-27','2023-02-28','2023-03-01','2023-03-02','2023-03-03','2023-03-06','2023-03-07','2023-03-08','2023-03-09','2023-03-10','2023-03-13','2023-03-14','2023-03-15','2023-03-16','2023-03-17','2023-03-20','2023-03-21','2023-03-22','2023-03-23','2023-03-24','2023-03-27','2023-04-03','2023-04-04','2023-04-05','2023-04-10','2023-04-11','2023-04-12','2023-04-13','2023-04-14','2023-04-25','2023-04-26','2023-04-27','2023-04-28','2023-05-03','2023-05-04','2023-05-05','2023-05-08','2023-05-09','2023-05-10','2023-05-11','2023-05-12','2023-05-15','2023-05-16',

Downloading: 100%|██████████| 978/978 [00:00<00:00, 1695.01rows/s]

Execução de query de viagens completas: sucesso.


In [95]:
viagem_completa_reprocessada = treat_trips(viagem_completa_reprocessada)
viagem_completa_reprocessada = check_trips(amostra_deduplicada, viagem_completa_reprocessada, "Viagem deferida")
print(viagem_completa_reprocessada.shape[0])
viagem_completa_reprocessada.to_excel('./../data/treated/viagem_completa_reprocessada.xlsx')

Não existem casos duplicados no cruzamento de dados.
981


In [96]:
viagem_conformidade_reprocessada = query_viagem_conformidade(data = datas, id_veiculo = id_veiculo, reprocessed=True,
                                                     reprocessed_table='rj-smtr-dev.SMTR202303002183_reprocessamento.viagem_conformidade')


  SELECT
    data,
    SUBSTRING(id_veiculo, 2) as id_veiculo,
    servico_informado,
    sentido,
    datetime_partida,
    datetime_chegada
  FROM
    rj-smtr-dev.SMTR202303002183_reprocessamento.viagem_conformidade
  WHERE 
    data IN ('2023-02-01','2023-02-02','2023-02-03','2023-02-06','2023-02-07','2023-02-08','2023-02-09','2023-02-10','2023-02-13','2023-02-14','2023-02-15','2023-02-16','2023-02-23','2023-02-24','2023-02-27','2023-02-28','2023-03-01','2023-03-02','2023-03-03','2023-03-06','2023-03-07','2023-03-08','2023-03-09','2023-03-10','2023-03-13','2023-03-14','2023-03-15','2023-03-16','2023-03-17','2023-03-20','2023-03-21','2023-03-22','2023-03-23','2023-03-24','2023-03-27','2023-04-03','2023-04-04','2023-04-05','2023-04-10','2023-04-11','2023-04-12','2023-04-13','2023-04-14','2023-04-25','2023-04-26','2023-04-27','2023-04-28','2023-05-03','2023-05-04','2023-05-05','2023-05-08','2023-05-09','2023-05-10','2023-05-11','2023-05-12','2023-05-15','2023-05-16','2023-05-17','2023

Downloading: 100%|██████████| 981/981 [00:00<00:00, 2265.65rows/s]

Execução de query de viagens conformidade: sucesso.


In [97]:
viagem_conformidade_reprocessada = treat_trips(viagem_conformidade_reprocessada)

viagem_conformidade_reprocessada = check_trips(viagem_completa_reprocessada, viagem_conformidade_reprocessada, "Viagem indeferida mesmo após o reprocessamento")
print(viagem_conformidade_reprocessada.shape[0])
viagem_conformidade_reprocessada.to_excel('./../data/treated/viagem_conformidade_reprocessada.xlsx')

Não existem casos duplicados no cruzamento de dados.
981


In [98]:
viagens_gps_reprocessada = gps_data(viagem_conformidade_reprocessada)
viagens_gps_reprocessada.to_excel('./../data/treated/viagem_gps_reprocessada.xlsx')

Downloading: 100%|██████████| 162/162 [00:00<00:00, 372.41rows/s]


Execução de query de posições de GPS: sucesso.
Acesso aos sinais de GPS concluído com sucesso.
Tratamento de dados de GPS concluído com sucesso.


In [99]:
contar_nan = viagens_gps_reprocessada['status'].notna().sum()
print(f"Tamanho da amostra: {amostra_tratada.shape[0]}")

print(f"Viagens classificadas: {contar_nan}")
print(f"Viagens classificadas em %: {round(contar_nan / amostra.shape[0],2)*100} %")

print(viagens_gps_reprocessada['status'].value_counts())

Tamanho da amostra: 981
Viagens classificadas: 981
Viagens classificadas em %: 100.0 %
Viagem deferida                                                                978
Sinal de GPS encontrado para o veículo operando no mesmo serviço da amostra      2
Viagem indeferida mesmo após o reprocessamento                                   1
Name: status, dtype: int64


In [100]:
# viagens_gps_reprocessada_start_end = check_start_end_gps(viagens_gps_reprocessada)
# viagens_gps_reprocessada_start_end.to_excel('./../data/treated/viagem_gps_reprocessada_start_end.xlsx')

Estes 2 casos em que o sinal de GPS foi encontrado para o veículo operando no mesmo serviço da amostra são as mesmas viagens do dia 31/05/2023 para o serviço 605, quando a linha ainda era chamada de LECD50. Ou seja, o serviço não foi planejado para o dia.

In [101]:
viagens_gps_reprocessada['status'] = viagens_gps_reprocessada['status'].replace(
    "Sinal de GPS encontrado para o veículo operando no mesmo serviço da amostra",
    "Viagem não planejada para o dia"
)

In [102]:
viagens_gps_reprocessada.to_excel('./../data/output/analise_amostra_pos_solucao.xlsx')